Importacion de librarias

In [31]:
import numpy as np
import pandas as pd
from pandas import *
import re
import math

Importando archivos de excel

In [32]:
file = ('segundoejercicio.xlsx')
file2 = ('butacas.xlsx')
repository_data = pd.read_excel(file, skiprows= 6, usecols=[0,1,2,3,4])
repository_butacas = pd.read_excel(file2)

Eliminando NAN (empty colums), eliminando las files que contienen totales

In [33]:
filter_valid_rows = repository_data.dropna(how='all')
filter_butacas = repository_butacas.dropna(how='all')
#remove totals
filter_valid_rows = filter_valid_rows.drop(filter_valid_rows[filter_valid_rows['Film'] == 'Total'].index)
filter_valid_rows = filter_valid_rows.drop(filter_valid_rows[filter_valid_rows['Screen'] == 'Total'].index)
filter_valid_rows = filter_valid_rows.drop(filter_valid_rows[filter_valid_rows['Cinema'] == 'Grand Total'].index)

LLenar los campos NAN de Screen y Cinema.

In [34]:
last_screen = 0
last_cinema = 0
remove_index_list = []
for index, row in filter_valid_rows.iterrows():
    #change NAN value
    try:
        screen = float( row['Screen'] )
        if( math.isnan( screen ) ):
            filter_valid_rows.at[index,'Screen'] = last_screen
        else:
            last_screen = screen

    except:
        remove_index_list = remove_index_list + [index]
   
    # remove cinema name. 
    #change NAN
    try:
        #remove text
        cinema = re.findall('[0-9]+', row['Cinema'])
        last_cinema = (cinema[0])
        filter_valid_rows.at[index,'Cinema'] = last_cinema
        
    except:
        filter_valid_rows.at[index,'Cinema'] = last_cinema
  

Eliminacion Subtitulos y calida de la pelicula. ( borrado de parentesis )

In [35]:
filter_valid_rows["Film"] = filter_valid_rows["Film"].str.split("\(").str.get(0)


remove NAN butacas file

In [36]:
#remove_index_list = []
#for index, row in repository_butacas.iterrows():
    #change NAN value
#    try:
#        cinema = float( row['Cinema'] )
#        if( math.isnan( cinema ) ):
#            remove_index_list = remove_index_list + [index]
#
#    except:
#        remove_index_list = remove_index_list + [index]
#repository_butacas = repository_butacas.drop(repository_butacas.index[remove_index_list])

Se agregan las butacas

In [37]:
repository_butacas
filter_valid_rows = filter_valid_rows.assign(butacas = 252 )

Se calcula la columna maxbutacas

In [38]:
#calculate maxButacas
filter_valid_rows['maxbutacas'] = filter_valid_rows['Number of Sessions'] * filter_valid_rows['butacas']

In [39]:
filter_valid_rows

,Cinema,Screen,Film,Number of Sessions,Admits,butacas,maxbutacas
0,2200,1,Batman,48.0,1830.0,252,12096.0
1,2200,1.0,Batman,18.0,419.0,252,4536.0
2,2200,1.0,Muestrame al Padre,1.0,84.0,252,252.0
3,2200,1.0,Los tipos Malos,5.0,51.0,252,1260.0
4,2200,1.0,Morbius,2.0,37.0,252,504.0
...,...,...,...,...,...,...,...
1388,795,10.0,Bachchhan Paandey,3.0,164.0,252,756.0
1389,795,10.0,El Exorcismo,5.0,46.0,252,1260.0
1390,795,10.0,Casate Conmigo,4.0,37.0,252,1008.0
1391,795,10.0,Morbius,3.0,16.0,252,756.0


Calculo de columna Occ 

In [58]:
movies = filter_valid_rows.groupby("Film").agg(sum)
movies.reset_index(inplace=True)
movies['Occ'] =  ( (movies['Admits'] / movies['maxbutacas'] ) *100 ).astype(int)

Eliminando Columnas en desuso

In [59]:
#remove colums
movies.drop(['Number of Sessions','butacas'], axis = 1, inplace = True) 

Agregando formato

In [60]:
movies = movies.astype({'Film': str, 'Admits': float, 'maxbutacas': float, 'Occ': str})

In [61]:
movies['Occ'] = movie

,Film,Admits,maxbutacas,Occ
0,Ambulance,852.0,10836.0,7
1,Ambulancia Plan de Huida,5300.0,111888.0,4
2,BTS Permission to Dance,6023.0,7560.0,79
3,Bachchhan Paandey,518.0,3276.0,15
4,Batman,208188.0,1392300.0,14
5,Belfast,1074.0,29232.0,3
6,Blacklight,513.0,10584.0,4
7,Blue Bayou,5.0,504.0,0
8,Caceria de Brujas,323.0,6552.0,4
9,Callejon de las almas perdidas,106.0,4536.0,2
